## A closer look at `UnaryOp`, `BinaryOp`, `Monoid`, and `Semiring` objects

This notebook was created for the effort to unify objects from `graphblas` and `pygraphblas` to `suitesparse-graphblas`.  This is the `graphblas` perspective.

For now, let's only consider builtin operations.  We'll consider user-defined objects (and user-defined types) later.

In [1]:
import graphblas as gb
import numpy as np

### First let's take a quick look at the "plus" binary operator.

In [2]:
plus = gb.binary.plus

In [3]:
type(plus)

graphblas.core.operator.BinaryOp

In [4]:
plus

binary.plus

In [5]:
type(plus[int])

graphblas.core.operator.TypedBuiltinBinaryOp

In [6]:
plus[int]

binary.plus[INT64]

In [7]:
plus[int] == plus["int"] == plus["INT64"] == plus[gb.dtypes.INT64] == plus[np.int64]

True

In [8]:
int in plus and "int64" in plus

True

<hr>

## `BinaryOp` attributes:
```python
name: str                 # Human-readable name
monoid: Optional[Monoid]  # Monoid of the same name (or None)
types: Dict[str, str]     # Input type to return type
```

In [9]:
plus.name

'plus'

In [10]:
plus.monoid is gb.monoid.plus

True

In [11]:
plus.types

{FC32: FC32,
 FC64: FC64,
 BOOL: BOOL,
 FP32: FP32,
 FP64: FP64,
 INT16: INT16,
 INT32: INT32,
 INT64: INT64,
 INT8: INT8,
 UINT16: UINT16,
 UINT32: UINT32,
 UINT64: UINT64,
 UINT8: UINT8}

<hr>

## `TypedBuiltinBinaryOp` attributes:
```python
name: str                             # Human-readable name
monoid: Optional[TypedBuiltinMonoid]  # Monoid of the same name and type (or None)
type: str                             # Input type
return_type: str                      # Return type
parent: BinaryOp                      # The "untyped" version of this BinaryOp
opclass: Literal["BinaryOp"]          # "BinaryOp" (used to determine which GraphBLAS function to call)
gb_name: str                          # Name of the GraphBLAS C object
gb_obj: cffi C data                   # The C object
```
`opclass`, `gb_name`, and `gb_obj` are (mostly) implementation details; easy to change their names or hide them.

In [12]:
plus[int].name

'plus'

In [13]:
plus[int].monoid == gb.monoid.plus[int]

True

In [14]:
plus[int].type

INT64

In [15]:
plus[int].return_type

INT64

In [16]:
plus[int].parent is plus

True

In [17]:
plus[int].opclass

'BinaryOp'

In [18]:
plus[int].gb_name

'GrB_PLUS_INT64'

In [19]:
plus[int].gb_obj

<cdata 'struct GB_BinaryOp_opaque *' 0x7f3dcd7e3780>

<hr>

`UnaryOp`s are just like `BinaryOp`s, but simpler (no `.monoid` attribute)

## `UnaryOp` attributes:
```python
name: str              # Human-readable name
types: Dict[str, str]  # Input type to return type
```

<hr>

## `TypedBuiltinUnaryOp` attributes:
```python
name: str                    # Human-readable name
type: str                    # Input type
return_type: str             # Return type
parent: UnaryOp              # The "untyped" version of this BinaryOp
opclass: Literal["UnaryOp"]  # "UnaryOp" (used to determine which GraphBLAS function to call)
gb_name: str                 # Name of the GraphBLAS C object
gb_obj: cffi C data          # The C object
```

In [20]:
exp = gb.unary.exp

In [21]:
type(exp)

graphblas.core.operator.UnaryOp

In [22]:
type(exp[float])

graphblas.core.operator.TypedBuiltinUnaryOp

In [23]:
exp[float] == exp["float"] == exp["FP64"] == exp[np.float64] == exp[gb.dtypes.FP64]

True

In [24]:
exp.name

'exp'

In [25]:
exp.types

{FC32: FC32,
 FC64: FC64,
 FP32: FP32,
 FP64: FP64,
 BOOL: FP32,
 INT8: FP32,
 INT16: FP32,
 UINT8: FP32,
 UINT16: FP32,
 INT32: FP64,
 INT64: FP64,
 UINT32: FP64,
 UINT64: FP64}

In [26]:
exp[float].opclass

'UnaryOp'

<hr>

## `Monoid` attributes:
```python
name: str                 # Human-readable name
binaryop: BinaryOp        # BinaryOp of the same name
types: Dict[str, str]     # Input type to return type (this could be a set!)
identities: Dict[str, T]  # The identity for each type (computed lazily)
```

In [27]:
plus = gb.monoid.plus

In [28]:
plus.name

'plus'

In [29]:
plus.binaryop == gb.binary.plus

True

In [30]:
plus.types

{FC32: FC32,
 FC64: FC64,
 FP32: FP32,
 FP64: FP64,
 INT16: INT16,
 INT32: INT32,
 INT64: INT64,
 INT8: INT8,
 UINT16: UINT16,
 UINT32: UINT32,
 UINT64: UINT64,
 UINT8: UINT8}

In [31]:
plus.identities

{FC32: 0j,
 FC64: 0j,
 FP32: 0.0,
 FP64: 0.0,
 INT16: 0,
 INT32: 0,
 INT64: 0,
 INT8: 0,
 UINT16: 0,
 UINT32: 0,
 UINT64: 0,
 UINT8: 0}

<hr>

## `TypedBuiltinMonoid` attributes:
```python
name: str                       # Human-readable name
binaryop: TypedBuiltinBinaryOp  # BinaryOp of the same name and type
type: str                       # Input type
return_type: str                # Return type (same as the input type)
identity: T                     # The identity value for this Monoid (computed lazily)
parent: Monoid                  # The "untyped" version of this Monoid
opclass: Literal["Monoid"]      # "Monoid" (used to determine which GraphBLAS function to call)
gb_name: str                    # Name of the GraphBLAS C object
gb_obj: cffi C data             # The C object
```

In [32]:
plus[int].identity

0

In [33]:
plus[float].identity

0.0

<hr>

## `Semiring` attributes:
```python
name: str              # Human-readable name
binaryop: BinaryOp     # The BinaryOp portion of the semiring
monoid: Monoid         # The Monoid portion of the semiring
types: Dict[str, str]  # Input type to return type
```

<hr>

## `TypedBuiltinSemiring` attributes:
```python
name: str                       # Human-readable name
binaryop: TypedBuiltinBinaryOp  # BinaryOp of the same name and type
monoid: TypedMonoid             # Monoid of the same name and type
type: str                       # Input type
return_type: str                # Return type (same as the input type)
parent: Semiring                # The "untyped" version of this Semiring
opclass: Literal["Semiring"]    # "Semiring" (used to determine which GraphBLAS function to call)
gb_name: str                    # Name of the GraphBLAS C object
gb_obj: cffi C data             # The C object
```

In [34]:
plus_land = gb.semiring.plus_land

In [35]:
plus_land.monoid is gb.monoid.plus

True

In [36]:
plus_land.binaryop is gb.binary.land

True

In [37]:
plus_land.types

{FP32: FP32,
 FP64: FP64,
 INT16: INT16,
 INT32: INT32,
 INT64: INT64,
 INT8: INT8,
 UINT16: UINT16,
 UINT32: UINT32,
 UINT64: UINT64,
 UINT8: UINT8}

<hr>

## On division

`C` and Python have different semantics for division.  See: https://github.com/cython/cython/wiki/enhancements-division

We renamed the `binary.div` to `binary.cdiv` and added `truediv` and `floordiv` that mimic Python semantics.  This way, it is explicit which semantics are used.  This should prevent errors whether the user is coming from C or Python.

We didn't do anything equivalent with `fmod`, `remainder`, or `rdiv`, but maybe we should.

In [38]:
gb.binary.cdiv[int].gb_name

'GrB_DIV_INT64'

In [39]:
gb.binary.truediv[int].gb_name  # reuse div, but cast int to float

'GrB_DIV_FP64'

In [40]:
gb.binary.floordiv[int].gb_name  # floordiv is a UDF

'floordiv_INT64'

In [41]:
# Semirings also updated
gb.semiring.plus_cdiv, gb.semiring.plus_truediv, gb.semiring.plus_floordiv

(semiring.plus_cdiv, semiring.plus_truediv, semiring.plus_floordiv)

<hr>

## On `graphblas.op` namespace

`graphblas.op` has all UnaryOps, BinaryOps, and Semirings.  This is incredibly convenient.
The methods in `graphblas` automatically convert `BinaryOp` to `Monoid` or vice versa where appropriate.

<hr>

## `graphblas` operators are callable
- Examples:
  - `exp(A)` apply
  - `plus(A, 1)` apply bind 2nd
  - `plus(A | B)` ewise_add
  - `plus(A & B)` ewise_mult
  - `plus_times(A @ B)` mxm, etc.
- So, we would need to make `suitesparse_graphblas` operators callable
  - Dispatch based on type of arguments; raise if none found
  - `graphblas` would need to register callback functions for its types

<hr>

# Possible enhancements

## Include commutation info for BinaryOps and Semirings
```python
>>> binary.le.commutes_to is binary.ge
True
>>> binary.plus.commutes_to is binary.plus
True
>>> binary.pow.commutes_to is None
True
>>> semiring.any_ge.commutes_to is semiring.any_le
True
```

## Include implied type coercion in `obj.types`
- For example, should `unary.exp` work on integer or bool types?
  - If so, should these be in `exp.types`?
    - Using numpy rules:
    ```python
{
    "BOOL": "FP32",  # "FP16" if existed
    "INT8": "FP32",  # "FP16" if existed
    "UINT8": "FP32",  # "FP16" if existed
    "INT16": "FP32",
    "UINT16": "FP32",
    "INT32": "FP64",
    "UINT32": "FP64",
    "INT64": "FP64",
    "UINT64": "FP64",
    ...
}
    ```
  - `exp[float]` can always be used explicitly

## Include types of left and right arguments in `obj.types`
- For example, user-defined functions may want different types for efficiency
- Very useful if need to support user-defined types: `(mytype, int)`
- No need to unify dtypes, e.g., `graphblas.dtypes.unify(type1, type2)`
- `first` binary operator on `(int, float)` types should probably return `int`

## Make operators `collections.abc.Mappings`
- For example, `isinstance(binary.plus, collections.abc.Mapping)`
- `binary.plus.items()` would iterate over `(type, TypedBuiltinBinaryOp)` pairs